In [ ]:
from pyedhrec import EDHRec
class MyEDHREc(EDHRec):
    def __init__(self):
        super().__init__()
        self._json_page_cards_url = "https://json.edhrec.com/pages/cards/"

    def get_similar_cards(self, cards_name: str):
        formatted_card_name = self.format_card_name(cards_name)
        url = f"{self._json_page_cards_url}{formatted_card_name}.json"
        data = self._get(url)
        return data["similar"]
myedhred = MyEDHREc()
similar_cards = myedhred.get_similar_cards("Pongify")
[x["name"] for x in similar_cards]

In [ ]:


import networkx as nx

import matplotlib.pyplot as plt
import gravis as gv
class Plotly_Graph_Ploter():
    def __init__(self):
        pass
    def assign_properties(self,g: nx.Graph):
        # Centrality calculation
        node_centralities = nx.in_degree_centrality(g)
        edge_centralities = nx.edge_betweenness_centrality(g)

        # Community detection
        communities = nx.algorithms.community.greedy_modularity_communities(g)

        # Graph properties
        g.graph['node_border_size'] = 1.5
        g.graph['node_border_color'] = 'white'
        g.graph['edge_opacity'] = 0.9

        # Node properties: Size by centrality, shape by size, color by community
        colors = ['red', 'blue', 'green', 'orange', 'pink', 'brown', 'yellow', 'cyan', 'magenta', 'violet']
        for node_id in g.nodes:
            node = g.nodes[node_id]
            node['size'] = 10 + node_centralities[node_id] * 100
            node['shape'] = 'circle' 
            for community_counter, community_members in enumerate(communities):
                if node_id in community_members:
                    break
            node['color'] = colors[community_counter % len(colors)]

        # Edge properties: Size by centrality, color by community (within=community color, between=black)
        for edge_id in g.edges:
            edge =  g.edges[edge_id]
            source_node = g.nodes[edge_id[0]]
            target_node = g.nodes[edge_id[1]]
            edge['size'] = edge_centralities[edge_id] * 100
            edge['color'] = source_node['color'] if source_node['color'] == target_node['color'] else 'black'


# Assign node and edge properties
    def plot_graph(self, G: nx.Graph):
        self.assign_properties(G)
        return gv.d3(G, edge_size_data_source='weight', use_edge_size_normalization=True)

In [ ]:
class SimilarityGraph():
    def __init__(self, graph: nx.Graph = nx.DiGraph()):
        self.graph = graph
        self.ploter = Plotly_Graph_Ploter()
        self.edhrec_asker = MyEDHREc()
    
    def add_card(self, card_name: str, depth_similar_cards: int = 1):
        if card_name in self.graph.nodes and self.graph.degree(card_name) > 0:
            pass
        similar_cards = self.edhrec_asker.get_similar_cards(card_name)
        for card in similar_cards:
            self.graph.add_edge(card_name, card["name"], weight=1)
            if depth_similar_cards > 1:
                self.add_card(card["name"], depth_similar_cards - 1)
    def plot_graph(self):
        return self.ploter.plot_graph(self.graph)
simi_graph = SimilarityGraph()
simi_graph.add_card("Tidal Barracuda", 3)
simi_graph.plot_graph()


In [ ]:
nx.eigenvector_centrality(simi_graph.graph,tol=0.1)

In [ ]:
dict(simi_graph.graph.adjacency())